# XGB Classifier with Filter-Based FS (Lexical + Content)

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [21]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_new_Bacud_unbalanced_lexical.csv")      # Loading the dataset

dataset.head()

,url_type,url_length,url_ip_in_domain,url_domain_entropy,url_is_digits_in_domain,url_query_length,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_is_https,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,16,0,3.169925,0,0,0,0,3.375000,0,...,0,0,0,0,0,0,0,0,0,0
1,0,35,0,2.807355,1,0,0,1,4.079143,0,...,0,0,0,0,0,0,0,0,0,0
2,0,31,0,2.921928,0,0,0,1,3.708093,0,...,0,0,0,0,0,0,0,0,0,0
3,1,88,0,2.896292,0,49,4,7,4.660343,0,...,0,0,1,0,0,0,0,0,0,27
4,1,235,0,3.405822,0,194,3,22,4.980518,0,...,0,0,1,0,0,0,0,0,0,27


In [22]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

In [23]:
valid = pd.read_csv("../../../02_feature-engineering/final-datasets/valid_unbalanced_with_lexical.csv")
valid.head()

y_valid = valid['url_type']
X_valid = valid.drop(columns=['url_type'])


#### 2. Preprocessing (Balancing)

In [24]:
dataset['url_type'].value_counts()

url_type
0    724778
1    380244
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [25]:
important_features_lexical = ['url_host_length',
                             'url_is_https',
                             'url_ip_in_domain',
                             'has_php_in_string',
                             'url_number_of_parameters',
                             'has_exe_in_string',
                             'url_has_port',
                             'url_is_digits_in_domain',
                             'url_path_length',
                             'url_num_question_mark', 
                             'url_query_length',
                             'url_string_entropy',
                             'url_num_periods',
                             'get_tld',
                             'url_scheme']

X_test_lexical = x_test[important_features_lexical]
X_train_lexical = x_train[important_features_lexical]
X_valid_lexical = X_valid[important_features_lexical]

#### Hyper-parameter Optimization

In [26]:
# Define the objective function for Optuna
def objective_lexical(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_lexical, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_lexical = optuna.create_study(direction='minimize')
study_lexical.optimize(objective_lexical, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_lexical = study_lexical.best_params
best_error_lexical = study_lexical.best_value
print("Best Hyperparameters (33 Features): ", best_params_lexical)
print("Best Error (33 Features): ", best_error_lexical)

[I 2024-04-26 00:22:01,620] A new study created in memory with name: no-name-21c74e6d-b1a1-4447-ad4b-da6de6cf0e6f
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:22:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653
[1]	validation-error:0.65653
[2]	validation-error:0.65653
[3]	validation-error:0.65653
[4]	validation-error:0.65653
[5]	validation-error:0.60550
[6]	validation-error:0.29766
[7]	validation-error:0.21205
[8]	validation-error:0.17096
[9]	validation-error:0.14587
[10]	validation-error:0.13073
[11]	validation-error:0.11768
[12]	validation-error:0.10770
[13]	validation-error:0.10229
[14]	validation-error:0.09820
[15]	validation-error:0.09195
[16]	validation-error:0.09002
[17]	validation-error:0.08745
[18]	validation-error:0.08630
[19]	validation-error:0.08578
[20]	validation-error:0.08525
[21]	validation-error:0.08421
[22]	validation-error:0.08341
[23]	validation-error:0.08310
[24]	validation-error:0.08297
[25]	validation-error:0.08244
[26]	validation-error:0.08271
[27]	validation-error:0.08278
[28]	validation-error:0.08287
[29]	validation-error:0.08263
[30]	validation-error:0.08257
[31]	validation-error:0.08270
[32]	validation-error:0.08223
[33]	validation-erro

[I 2024-04-26 00:22:19,069] Trial 0 finished with value: 0.2758952470981087 and parameters: {'eta': 0.09625392176813431, 'max_depth': 7, 'subsample': 0.7063007340659195, 'colsample_bytree': 0.5525493183220774, 'gamma': 9.307147005921308, 'min_child_weight': 3.7829127861473086, 'lambda': 1.8996565250925959, 'alpha': 5.93840452551558}. Best is trial 0 with value: 0.2758952470981087.


[0]	validation-error:0.65653


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:22:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


...

[1]	validation-error:0.26010
[2]	validation-error:0.12736
[3]	validation-error:0.10827
[4]	validation-error:0.09128
[5]	validation-error:0.08574
[6]	validation-error:0.08193
[7]	validation-error:0.08023
[8]	validation-error:0.07949
[9]	validation-error:0.07933
[10]	validation-error:0.07916
[11]	validation-error:0.07832
[12]	validation-error:0.07790
[13]	validation-error:0.07733
[14]	validation-error:0.07710
[15]	validation-error:0.07692
[16]	validation-error:0.07647
[17]	validation-error:0.07633
[18]	validation-error:0.07617
[19]	validation-error:0.07614
[20]	validation-error:0.07613
[21]	validation-error:0.07610
[22]	validation-error:0.07592
[23]	validation-error:0.07576
[24]	validation-error:0.07567
[25]	validation-error:0.07567
[26]	validation-error:0.07563
[27]	validation-error:0.07550
[28]	validation-error:0.07545
[29]	validation-error:0.07512
[30]	validation-error:0.07500
[31]	validation-error:0.07517
[32]	validation-error:0.07501
[33]	validation-error:0.07495
[34]	validation-err

[I 2024-04-26 00:35:58,993] Trial 98 pruned. Trial was pruned at iteration 77.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:35:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653


[I 2024-04-26 00:35:59,618] Trial 99 pruned. Trial was pruned at iteration 1.


Best Hyperparameters (33 Features):  {'eta': 0.2894936852269883, 'max_depth': 10, 'subsample': 0.8149353457589634, 'colsample_bytree': 0.8217748861840777, 'gamma': 1.1641475399125378, 'min_child_weight': 0.3484513223924258, 'lambda': 5.7585548659394075, 'alpha': 4.983554818204054}
Best Error (33 Features):  0.2688031856179489


#### Model Training

In [28]:
from sklearn.model_selection import KFold
from xgboost import DMatrix, train

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

best_params_lexical['objective'] = 'binary:hinge'
best_params_lexical['eval_metric'] = 'error'

# Convert the data into DMatrix format
lexical_train = DMatrix(X_train_lexical, label=y_train)
lexical_test = DMatrix(X_test_lexical, label=y_test)
lexical_valid = DMatrix(X_valid_lexical, label = y_valid)

# Train the Model
xgb_classifier_lexical = train(best_params_lexical, lexical_train, num_boost_round=3000)
y_pred_lexical = xgb_classifier_lexical.predict(lexical_test)
y_pred_valid = xgb_classifier_lexical.predict(lexical_valid)

print("Model training done.")

Model training done.


In [29]:
# Classification Report
print("------------- Test Evaluation -------------")
print(classification_report(y_test, y_pred_lexical))


------------- Test Evaluation -------------
              precision    recall  f1-score   support

           0       0.93      0.96      0.94    144867
           1       0.91      0.87      0.89     76138

    accuracy                           0.93    221005
   macro avg       0.92      0.91      0.92    221005
weighted avg       0.93      0.93      0.93    221005



In [31]:
# Classification Report
print("------------- Practical Evaluation -------------")

print(classification_report(y_valid, y_pred_valid))

------------- Practical Evaluation -------------
              precision    recall  f1-score   support

         0.0       1.00      0.62      0.77      1000
         1.0       0.00      0.00      0.00         0

    accuracy                           0.62      1000
   macro avg       0.50      0.31      0.38      1000
weighted avg       1.00      0.62      0.77      1000



c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

#### Practical Evaluation

In [ ]:
# Dumping the model
joblib.dump(xgb_classifier_lexical, 'xgb_filter_lexical.sav')

In [34]:
import lexical_generator_filter_lexical
import time

def xgb_predict_maliciousness(url):

    numerical_values = lexical_generator_filter_lexical.lexical_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_lexical.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "youtube.com"
print("Current URL: "+url)

print("------------- Filter-Based (Lexical) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

Current URL: youtube.com
------------- Filter-Based (Lexical) -------------
Trial 0
Malware
0.021596700069494545
Trial 1
Malware
0.028116500005126
Trial 2
Malware
0.029835800058208406
Trial 3
Malware
0.026454199920408428
Trial 4
Malware
0.026657800073735416
Trial 5
Malware
0.026782599976286292
Trial 6
Malware
0.026602900004945695
Trial 7
Malware
0.025368999922648072
Trial 8
Malware
0.02986169990617782
Trial 9
Malware
0.02698920003604144
Trial 10
Malware
0.025413399911485612
Trial 11
Malware
0.02379460004158318
Trial 12
Malware
0.027031300007365644
Trial 13
Malware
0.024985099909827113
Trial 14
Malware
0.02613999997265637


#### Evaluation

In [ ]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred, labels=xgb_classifier.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier.classes_)
disp.plot()
plt.show()'''

In [ ]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''